<a href="https://colab.research.google.com/github/meenarahmanzai/data-analysis-portfolio/blob/main/sanfrancisco_bikeshare_casestudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ipython-sql sqlalchemy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.0 MB/s eta 0:00:00


In [3]:
%load_ext sql
%sql sqlite:///bikeshare.db

In [7]:
!pip install --upgrade google-cloud-bigquery pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.1/240.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 72.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.25.0
    Uninstalling google-cloud-bigquery-3.25.0:
      Successfully uninstalled google-cloud-bigquery-3.25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [13]:
from google.colab import auth
auth.authenticate_user()

In [20]:
from google.cloud import bigquery
import pandas as pd

In [15]:
client = bigquery.Client()

# San Francisco BikeShare Dataset Analysis
## Business Task
-- The goal of this analysis is to come up with a strategy on how to get customers to become subsribers. To do this, we will evaluate our customers and subscribers bike usage patterns and which stations are most popular.

-- First we will analyze the number of trips taken per day.

In [23]:
query = """
SELECT
  DATE(start_date) AS trip_date,
  COUNT(*) AS trip_count
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
GROUP BY
  trip_date
ORDER BY
  trip_date DESC;
"""

-- Next, we will analyze the number of trips taken per month and order by trip_count DESC to view which months were the most popular.

In [28]:
query = """
SELECT
  EXTRACT(YEAR FROM start_date) AS year,
  EXTRACT(MONTH FROM start_date) AS month,
  COUNT(*) AS trip_count
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
GROUP BY
  year, month
ORDER BY
  trip_count DESC;SELECT
  EXTRACT(YEAR FROM start_date) AS year,
  EXTRACT(MONTH FROM start_date) AS month,
  COUNT(*) AS trip_count
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
GROUP BY
  year, month
ORDER BY
  trip_count DESC;
"""

-- Now we will analyze the average trip duration by the hour of the day. This will help identify if certain times of the day have shorter or longer bike trips, what the peak times are.

In [30]:
query = """
SELECT
  EXTRACT(HOUR FROM start_date) AS hour_of_day,
  AVG(duration_sec) AS avg_duration_sec
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
GROUP BY
  hour_of_day
ORDER BY
  avg_duration_sec DESC;
"""

-- From this we can see that 3 am, 2am, and 1 am are the times our bikes are used the longest. We can infer that people rely on our bikes to get home after public transport lines stop.

-- Now we will query the most popular bike stations based on the number of trips that start or end there.

In [31]:
query = """
SELECT
  start_station_name,
  COUNT(*) AS start_trip_count
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
GROUP BY
  start_station_name
ORDER BY
  start_trip_count DESC
LIMIT 10;
"""

In [ ]:
query = """
SELECT
  end_station_name,
  COUNT(*) AS end_trip_count
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
GROUP BY
  end_station_name
ORDER BY
  end_trip_count DESC
LIMIT 10;
"""

-- From the results we can concur that the top 3 popular start and end stations are San Francisco Caltrain, San Francisco Caltrain 2, and Harry Bridges Plaza. The top 3 popular start and end stations being the same stations tell us that users most likely return the bikes from the same place they rent it from.

-- Now, we will compare the average trip duration by user type (e.g., "Subscriber" vs. "Customer").

In [32]:
query = """
SELECT
  subscriber_type,
  AVG(duration_sec) AS avg_duration_sec
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
GROUP BY
  subscriber_type
ORDER BY
  avg_duration_sec DESC;
"""

-- From this query we come across another subsriber type "nan". After taking a look at the table again, it seems there is another column named c_subscription_type that indicated what kind of user the rider was where the subscriber_type column doesn't. To get a more accurate result, we will combine the columns so that the data is uniform and make it a new column named combined_subscription_type.

In [33]:
query = """
SELECT
  combined_subscription_type,
  AVG(duration_sec) AS avg_duration_sec
FROM
  (
    SELECT
      IF(subscriber_type = 'nan', c_subscription_type, subscriber_type) AS combined_subscription_type,
      duration_sec
    FROM
      `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`)
GROUP BY
  combined_subscription_type
ORDER BY
  avg_duration_sec DESC;
"""

-- From this we can see that on average, customers use our bikes for longer trips than subscribers.

-- Now that we know which user type uses our bikes for a longer duration, we will now analyze which user type takes more trips on average. We will be using the combined column we made for accurate results.

In [34]:
query = """
WITH combined_data AS (
  SELECT
    IF(subscriber_type = 'nan', c_subscription_type, subscriber_type) AS combined_subscription_type,
    duration_sec
  FROM
    `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
)

SELECT
  combined_subscription_type,
  COUNT(*) AS total_trips,
  SUM(duration_sec) AS total_duration_sec,
  AVG(duration_sec) AS avg_trip_duration_sec
FROM
  combined_data
GROUP BY
  combined_subscription_type
ORDER BY
  total_trips DESC;
"""

-- From this we can concur that subscribers took more trips than customers.

# Conclusion
-- After analyzing the data, we conclude that while customers tend to use our bikes for longer trips, subscribers take a significantly higher number of trips overall. Additionally, the top three most popular start and end stations are San Francisco Caltrain, San Francisco Caltrain 2, and Harry Bridges Plaza. To encourage more customers to become subscribers, we could implement a points system where riders earn points for each trip taken or distance ridden. We could also offer flexible subscription options, for example, a Pay-per-mile subscription option for long-distance riders. Furthermore, since the top three start stations are in high demand, ensuring bike availability is crucial for a positive user experience. We could implement a "Station Full" notification feature on our app to redirect riders to nearby stations when a station is full.